In [21]:
import os
import sys
import pickle
import numpy as np
#from warnings import filterwarnings
#filterwarnings('ignore')
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error

In [20]:
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from bayesmf.models.nmf import VanillaNMF, ConsensusNMF
from bayesmf.models.lda import LDA, StochasticLDA
from bayesmf.models.bmf import BMF, StochasticBMF
from bayesmf.models.cmf import CMF, StochasticCMF
from bayesmf.scripts.utils.model import workhorse, run_kfold_xval

%reload_ext autoreload
%autoreload 2

In [23]:
import pprint

In [26]:
files = ['errs_050720.pkl','errs_nmf_050720.pkl','d_errs_050720.pkl','d_errs_nmf_050720.pkl']
for file in files:
    print(file)
    errs = pickle.load(open(os.path.join('/home/sxchao/bayesmf/output',file), 'rb'))
    pprint.pprint(errs)

errs_050720.pkl
{10: {'bmf-batch': [1.307895418251388,
                    1.322977266917406,
                    1.2759307326649187,
                    1.3235378908612179,
                    1.3485933812382755,
                    1.3506370257279103,
                    1.3792402884327568,
                    1.340321254283131,
                    1.296426170400274,
                    1.2592109331493488],
      'bmf-stochastic': [1.2413429035483325,
                         1.24789596344033,
                         1.232532105399029,
                         1.2782248588288216,
                         1.3073118011153209,
                         1.3318493034588148,
                         1.3328640692529148,
                         1.3037718814377466,
                         1.2467235108833206,
                         1.2336521015872455],
      'cmf-batch': [1.5191403896149838,
                    1.5548094670649844,
                    1.5000918034576805,
                   

In [3]:
alpha = 2   # CRP parameter
D = 1000    # number of datapoints
K = 15      # number of (latent) topics
V = 100     # number of features

In [4]:
# simulate a chinese restaurant process
np.random.seed(22690)

seating_chart = [[0]]
num_tables = 1
for d in range(1, D):
    probs = [len(x) for x in seating_chart] + [alpha]
    probs /= sum(np.array(probs))

    table_assignment = np.random.choice(np.arange(num_tables + 1), p=probs)

    if table_assignment == num_tables:
        seating_chart.append([d])
        num_tables += 1
    else:
        seating_chart[table_assignment].append(d)

num_tables

13

In [13]:
# sample the topic-specific feature weights
feature_weights = np.random.multivariate_normal(np.zeros(V) - 0.5, np.eye(V), K).T
feature_weights[feature_weights < 0.0] = 0.0
#feature_weights[feature_weights < 0.0] = -feature_weights[feature_weights < 0.0]
feature_weights.shape

(100, 15)

In [14]:
# sample the table-specific topic weights
topic_weights = np.random.multivariate_normal(np.zeros(K) - 0.5, np.eye(K), num_tables).T
topic_weights[topic_weights < 0.0] = 0.0
#topic_weights[topic_weights < 0.0] = -topic_weights[topic_weights < 0.0]
topic_weights.shape

(15, 13)

In [15]:
# sample the occupant-specific gene expression
sigma = np.eye(K)

lambd = []
for c,occupants in enumerate(seating_chart):
    mu = topic_weights[:, c]
    weights = np.random.multivariate_normal(mu, sigma, len(occupants)).T
    weights[weights < 0.0] = 0.0    
    lambd.append(np.matmul(feature_weights, weights))

lambd = np.concatenate(lambd, axis=1)
X = np.random.poisson(lambd)
X.shape

(100, 1000)

In [19]:
# sparsity
np.count_nonzero(X==0) / X.size

0.37067

In [126]:
K=15
ms=[10]
num_stepss=[1, 2]
step_sizes=[1e-15, 1e-10, 1e-05]
max_iterss=[100] 
tolerances=[5e-04]
smoothnesss=[100]

In [127]:
idxs = np.arange(X.shape[1])
np.random.seed(22690)
np.random.shuffle(idxs)
X_train = X[:, idxs[200:]]
X_test = X[:, idxs[:200]]

In [128]:
# cmf-stochastic
print('m ns ssize it tol sm rmse')
for m in ms:
    for num_steps in num_stepss:
        for step_size in step_sizes:
            for max_iters in max_iterss:
                for tolerance in tolerances:
                    for smoothness in smoothnesss:
                        factorizer = StochasticCMF(minibatch_size=100, K=K, m=m, num_steps=num_steps, max_iters=max_iters, tolerance=tolerance, smoothness=smoothness, kwargs={'c0':0.05 * X_train.shape[0]})
                        factorizer.fit(X_train) # V x D
                        l = factorizer.transform(X_test, attr='l') # K x m
                        W = np.exp(factorizer.alpha[np.newaxis, :] + np.dot(l, factorizer.u.T)).T # K x D -> D x K
                        H = factorizer.Eb.T # V x K -> K x D
                        rmse = mean_squared_error(X_test.T, np.matmul(W, H), squared=False)
                        print(m, num_steps, step_size, max_iters, tolerance, smoothness, rmse)

m ns ssize it tol sm rmse
10 1 1e-15 100 0.0005 100 1.5379893662897248
10 1 1e-10 100 0.0005 100 1.5379893662897248
10 1 1e-05 100 0.0005 100 1.5379893662897248
10 2 1e-15 100 0.0005 100 1.5385629783510322
10 2 1e-10 100 0.0005 100 1.5385629783510322
10 2 1e-05 100 0.0005 100 1.5385629783510322


In [46]:
# cmf-batch

m ns ssize it tol sm rmse
10 50 1e-05 100 0.0005 500 1.5372329151598225
10 50 1e-05 100 0.0005 400 1.5372329149036377
10 50 1e-05 100 0.0005 300 1.5372329905406632
10 40 1e-05 100 0.0005 500 1.5372329535140388
10 40 1e-05 100 0.0005 400 1.537232953029216
10 40 1e-05 100 0.0005 300 1.5372330890043964
10 30 1e-05 100 0.0005 500 1.537233012134699
10 30 1e-05 100 0.0005 400 1.5372330112120565
10 30 1e-05 100 0.0005 300 1.5372332419447163
15 50 1e-05 100 0.0005 500 1.537233163680542
15 50 1e-05 100 0.0005 400 1.5372331643239827
15 50 1e-05 100 0.0005 300 1.5372326668878677
15 40 1e-05 100 0.0005 500 1.5372333995824745
15 40 1e-05 100 0.0005 400 1.5372334007755923
15 40 1e-05 100 0.0005 300 1.5372325088669738
15 30 1e-05 100 0.0005 500 1.5372337628444517
15 30 1e-05 100 0.0005 400 1.537233765089291
15 30 1e-05 100 0.0005 300 1.5372322672086436


In [212]:
X_train = np.random.poisson(2, (100, 1000))

In [229]:
bmf = CMF(K=15, init='nmf', verbose=True)

In [230]:
bmf.fit(X_train[:,:800])

Iter: 0, Bound: 6464893.78, Change: nan
Iter: 1, Bound: 6767331.91, Change: 0.04678
Iter: 2, Bound: 6813977.80, Change: 0.00689
Iter: 3, Bound: 6809156.37, Change: -0.00071


CMF(K=15, init='nmf', m=10, max_iters=100, num_steps=1, random_state=22690,
    smoothness=100, step_size=1e-05, tolerance=0.0005, verbose=True)

In [231]:
l = bmf.transform(X_train[:,800:], attr='l') # K x m
W = np.exp(bmf.alpha[np.newaxis, :] + np.dot(l, bmf.u.T)).T # K x D -> D x K
H = bmf.Eb.T

Iter: 0, Bound: 1698737.73, Change: nan
Iter: 1, Bound: 1698738.77, Change: 0.00000


In [232]:
# w/ nmf
X_pred = np.matmul(W, H)
mean_squared_error(X_test.T, X_pred, squared=False)

1.921133682095794

In [222]:
# w/o nmf
X_pred = np.matmul(W, H)
mean_squared_error(X_test.T, X_pred, squared=False)

1.9211048773757262

In [208]:
errs, durs = run_kfold_xval(X, kfold=10, components=[20], methods=['bmf-batch', 'bmf-stochastic'])

Method: bmf-batch, RMSE: 1.075, Dur: 0.652
Method: bmf-stochastic, RMSE: 1.083, Dur: 6.966



Method: bmf-batch, RMSE: 1.078, Dur: 0.744
invalid algorithm for bmf


UnboundLocalError: local variable 'W' referenced before assignment

In [207]:
errs, durs = run_kfold_xval(X, kfold=10, components=[20], methods=['bmf-batch', 'bmf-stochastic'])

Method: bmf-batch, RMSE: 1.072, Dur: 0.544
Method: bmf-stochastic, RMSE: 1.079, Dur: 6.654



Method: bmf-batch, RMSE: 1.074, Dur: 0.661
invalid algorithm for bmf


UnboundLocalError: local variable 'W' referenced before assignment

In [133]:
errs, durs = run_kfold_xval(X, kfold=10, components=[20])

Method: nmf-vanilla, RMSE: 1.023, Dur: 0.313
Method: nmf-consensus, RMSE: 1.031, Dur: 18.536
Method: lda-batch, RMSE: 2.270, Dur: 12.494
Method: lda-stochastic, RMSE: 2.269, Dur: 15.924
Method: bmf-batch, RMSE: 1.210, Dur: 0.425
Method: bmf-stochastic, RMSE: 1.125, Dur: 2.626
Method: cmf-batch, RMSE: 1.519, Dur: 1.289
invalid algorithm for cmf


UnboundLocalError: local variable 'W' referenced before assignment

In [132]:
errs, durs = run_kfold_xval(X, components=[20])

Method: nmf-vanilla, RMSE: 1.022, Dur: 0.310
Method: nmf-consensus, RMSE: 1.028, Dur: 16.328
Method: lda-batch, RMSE: 2.278, Dur: 11.666
Method: lda-stochastic, RMSE: 2.277, Dur: 16.306
Method: bmf-batch, RMSE: 1.202, Dur: 0.471
Method: bmf-stochastic, RMSE: 1.121, Dur: 2.377
Method: cmf-batch, RMSE: 1.537, Dur: 1.262
invalid algorithm for cmf


UnboundLocalError: local variable 'W' referenced before assignment

In [131]:
errs, durs = run_kfold_xval(X, components=[15])

Method: nmf-vanilla, RMSE: 1.087, Dur: 0.143
Method: nmf-consensus, RMSE: 1.103, Dur: 12.317
Method: lda-batch, RMSE: 2.278, Dur: 12.177
Method: lda-stochastic, RMSE: 2.277, Dur: 17.308
Method: bmf-batch, RMSE: 1.246, Dur: 0.430
Method: bmf-stochastic, RMSE: 1.169, Dur: 2.188
Method: cmf-batch, RMSE: 1.537, Dur: 1.041
invalid algorithm for cmf


UnboundLocalError: local variable 'W' referenced before assignment

In [130]:
errs, durs = run_kfold_xval(X, components=[10])

Method: nmf-vanilla, RMSE: 1.197, Dur: 0.149
Method: nmf-consensus, RMSE: 1.200, Dur: 8.581
Method: lda-batch, RMSE: 2.278, Dur: 10.957
Method: lda-stochastic, RMSE: 2.277, Dur: 16.131
Method: bmf-batch, RMSE: 1.332, Dur: 0.323
Method: bmf-stochastic, RMSE: 1.262, Dur: 2.047
Method: cmf-batch, RMSE: 1.537, Dur: 0.717
invalid algorithm for cmf


UnboundLocalError: local variable 'W' referenced before assignment

In [ ]:
'''V = 100
K = 15
m = 14
D = 1000
X = np.random.poisson(0.7, size=(V, D))
np.count_nonzero(X==0) / X.size

cmf = CMF(K=K, m=m, step_size=0.000001, verbose=True, kwargs={'c0':0.05*V})
cmf.fit(X)

l = cmf.l
W = (cmf.alpha[np.newaxis, :] + np.dot(l, cmf.u.T)).T # K x D -> D x K
H = cmf.Eb.T

mean_squared_error(X.T, np.matmul(W, H), squared=False)

scmf = StochasticCMF(K=K, m=m, minibatch_size=D, verbose=True, kwargs={'kappa':0.0})
scmf.fit(X)'''